In [1]:
%run -i 'ModelParsing.py'

# 1

Organism: Piscirickettsia salmonis

Version: iPS584 and iPSA590

Year:2018

Paper link: https://www.frontiersin.org/articles/10.3389/fmicb.2017.02462/full

Model Link: http://psalmonis.cmm.uchile.cl/models/

In [2]:
model =parseSBML('/home/hzabeti/Desktop/UCSD models/Psalmonis_genome_scale_models/iPS584.xml')

0) P. salmonis biomass objective function 1) P. salmonis biomass objective function without ions
Please select the correct string from the list above and enter its number, or -1 if none 
0


In [3]:
model.description

{'ID': 'iPS584',
 'Name': '',
 'Source File': '/home/hzabeti/Desktop/UCSD models/Psalmonis_genome_scale_models/iPS584.xml'}

In [9]:
getSize(model.fullMatrix)

(1114, 1323)

In [8]:
flm_index_list,flm_list = model.findFreeLunchMetabolites()

No Free Lunch condition: Satisfied!


In [2]:
model =parseSBML('/home/hzabeti/Desktop/UCSD models/Psalmonis_genome_scale_models/iPSA590.xml')

ValueError: 'M_cu_p' is not in list

# 2

Organism: Streptomyces clavuligerus	

Version: iLT1021

Year: 2018

Paper link: https://link.springer.com/article/10.1007%2Fs00449-018-1900-9

Model Link: https://static-content.springer.com/esm/art%3A10.1007%2Fs00449-018-1900-9/MediaObjects/449_2018_1900_MOESM2_ESM.xml

In [3]:
model =parseSBML('/home/hzabeti/Desktop/UCSD models/12918_2008_242_MOESM5_ESM.XML')

In [4]:
getSize(model.fullMatrix)

(828, 996)

In [5]:
flm_index_list,flm_list = model.findFreeLunchMetabolites()

No Free Lunch condition: Not satisfied!
Optimum solution and the number of all possible free lunch metabolites 2


In [6]:
print(flm_index_list,flm_list)

[315, 335] ['M_PROTON_Periplasm', 'M_PROTON_Cytosol']


In [7]:
ind_flm_index_list,ind_flm_list = model.findIndividuallyProducibleFLM()

Number of individually producible free lunch metabolites 2


In [8]:
minimal_free_lunch,free_lunch_list,free_lunch_meta = model.findMinimalFreeLunch(free_lunch_meta_list=[315])

Number of iteration:  0
Number of reactions involved in FL: 5
------------------------------------------------------------
Checking index:  18
This index is in the minimal set 18
Checking index:  201
This index is in the minimal set 201
Checking index:  314
This index is in the minimal set 314
Checking index:  325
This index is in the minimal set 325
Checking index:  542
This index is in the minimal set 542
Minimal free lunch indexes: [18, 201, 314, 325, 542]
------------------ minimal free lunch equation----------------
3/4*(1 M_WATER_Cytosol[Cytosol] + 1 M_ATP_Cytosol[Cytosol] -> 1 M_PI_Cytosol[Cytosol] + 1 M_PROTON_Cytosol[Cytosol] + 1 M_ADP_Cytosol[Cytosol])
-1/2*(1 M_WATER_Cytosol[Cytosol] + 1 M_ATP_Cytosol[Cytosol] -> 1 M_AMP_Cytosol[Cytosol] + 1 M_PPI_Cytosol[Cytosol])
-1/4*(1 M_PI_Cytosol[Cytosol] + 4 M_PROTON_Periplasm[Periplasm] + 1 M_ADP_Cytosol[Cytosol] <-> 3 M_PROTON_Cytosol[Cytosol] + 1 M_WATER_Cytosol[Cytosol] + 1 M_ATP_Cytosol[Cytosol])
-1/2*(1 M_PPI_Cytosol[Cytosol] + 

In [9]:
flw_index_list, y_lower_bound = model.findFreeLunchWitnesses()

parameters.mip.tolerances.integrality
CPXPARAM_Simplex_Tolerances_Feasibility          1.0000000000000001e-09
CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Tolerances_Integrality              1.0000000000000001e-15
Found incumbent of value 230.000000 after 0.00 sec. (0.16 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 312 rows and 279 columns.
MIP Presolve modified 1245 coefficients.
Aggregator did 18 substitutions.
Reduced MIP has 1418 rows, 1405 columns, and 7203 nonzeros.
Reduced MIP has 797 binaries, 4 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (6.20 ticks)
Probing time = 0.00 sec. (0.17 ticks)
Tried aggregator 1 time.
Reduced MIP has 1418 rows, 1405 columns, and 7203 nonzeros.
Reduced MIP has 797 binaries, 4 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (2.91 ticks)
Probing time = 0.00 sec. (0.17 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, us

# 3

Organism: Piscirickettsia salmonis

Version: iPF215

Year: 2016

Paper link: https://www.sciencedirect.com/science/article/pii/S0960852416314304?via%3Dihub

Model Link: https://ars.els-cdn.com/content/image/1-s2.0-S0960852416314304-mmc2.xml

In [11]:
model =parseSBML('/home/hzabeti/Desktop/UCSD models/1-s2.0-S0960852416314304-mmc2.xml')

In [12]:
getSize(model.fullMatrix)

(449, 476)

In [13]:
flm_index_list,flm_list = model.findFreeLunchMetabolites()

No Free Lunch condition: Not satisfied!
Optimum solution and the number of all possible free lunch metabolites 3


In [14]:
print(flm_index_list,flm_list)

[152, 375, 446] ['M_cpd00067_e0_c', 'M_cpd00067_c0_c', 'M_cpd00067_e0_c_b']


In [15]:
ind_flm_index_list,ind_flm_list = model.findIndividuallyProducibleFLM()

Number of individually producible free lunch metabolites 3


In [16]:
minimal_free_lunch,free_lunch_list,free_lunch_meta = model.findMinimalFreeLunch(free_lunch_meta_list=[152])

Number of iteration:  0
Number of reactions involved in FL: 4
------------------------------------------------------------
Checking index:  49
This index is in the minimal set 49
Checking index:  106
This index is in the minimal set 106
Checking index:  302
This index is in the minimal set 302
Checking index:  329
This index is in the minimal set 329
Minimal free lunch indexes: [49, 106, 302, 329]
------------------ minimal free lunch equation----------------
-1*(1 M_cpd00072_c0_c[c] + 1 M_cpd00002_c0_c[c] -> 1 M_cpd00008_c0_c[c] + 1 M_cpd00290_c0_c[c])
3/4*(1 M_cpd00002_c0_c[c] + 1 M_cpd00001_c0_c[c] -> 1 M_cpd00008_c0_c[c] + 1 M_cpd00009_c0_c[c] + 1 M_cpd00067_c0_c[c])
-1*(1 M_cpd00290_c0_c[c] + 1 M_cpd00001_c0_c[c] -> 1 M_cpd00072_c0_c[c] + 1 M_cpd00009_c0_c[c])
-1/4*(1 M_cpd00008_c0_c[c] + 1 M_cpd00009_c0_c[c] + 4 M_cpd00067_e0_c[c] -> 1 M_cpd00002_c0_c[c] + 1 M_cpd00001_c0_c[c] + 3 M_cpd00067_c0_c[c])
-----------
1*(M_cpd00067_e0_c)
------------------------------------------------

In [17]:
flw_index_list, y_lower_bound = model.findFreeLunchWitnesses()

parameters.mip.tolerances.integrality
CPXPARAM_Simplex_Tolerances_Feasibility          1.0000000000000001e-09
CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Tolerances_Integrality              1.0000000000000001e-15
Found incumbent of value 204.000000 after 0.00 sec. (0.09 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 162 rows and 165 columns.
MIP Presolve modified 442 coefficients.
Aggregator did 4 substitutions.
Reduced MIP has 784 rows, 755 columns, and 4269 nonzeros.
Reduced MIP has 408 binaries, 1 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (3.60 ticks)
Probing time = 0.00 sec. (0.08 ticks)
Tried aggregator 1 time.
Reduced MIP has 784 rows, 755 columns, and 4269 nonzeros.
Reduced MIP has 408 binaries, 1 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (1.68 ticks)
Probing time = 0.00 sec. (0.08 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up

# 4

Organism: Ralstonia solanacearum

Version: iRP1476

Year: 2016

Paper link: https://journals.plos.org/plospathogens/article?id=10.1371/journal.ppat.1005939

Model Link: http://lipm-bioinfo.toulouse.inra.fr/systemsbiology/models/rsolanacearum/data/metabolism/sbml/1.0/iRP1476.xml

In [41]:
model =parseSBML('/home/hzabeti/Desktop/UCSD models/iRP1476.xml')

In [42]:
model.description

{'ID': 'iRP1476',
 'Name': 'Ralstonia solanacearum GMI1000 iRP1476',
 'Source File': '/home/hzabeti/Desktop/UCSD models/iRP1476.xml'}

In [43]:
getSize(model.fullMatrix)

(2574, 2644)

In [44]:
flm_index_list,flm_list = model.findFreeLunchMetabolites()

No Free Lunch condition: Satisfied!


In [45]:
ind_flm_index_list,ind_flm_list = model.findIndividuallyProducibleFLM()

Number of individually producible free lunch metabolites 0


In [46]:
minimal_free_lunch,free_lunch_list,free_lunch_meta = model.findMinimalFreeLunch()

In [48]:
print(minimal_free_lunch)

[]


In [47]:
flw_index_list, y_lower_bound = model.findFreeLunchWitnesses()

parameters.mip.tolerances.integrality
CPXPARAM_Simplex_Tolerances_Feasibility          1.0000000000000001e-09
CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Tolerances_Integrality              1.0000000000000001e-15
Found incumbent of value 1224.000000 after 0.06 sec. (0.59 ticks)
Tried aggregator 3 times.
MIP Presolve eliminated 2176 rows and 2089 columns.
MIP Presolve modified 9076 coefficients.
Aggregator did 17 substitutions.
Reduced MIP has 3093 rows, 3109 columns, and 21209 nonzeros.
Reduced MIP has 1842 binaries, 10 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.09 sec. (50.91 ticks)
Probing time = 0.00 sec. (0.26 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 8 rows and 8 columns.
Reduced MIP has 3085 rows, 3101 columns, and 21185 nonzeros.
Reduced MIP has 1838 binaries, 10 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.05 sec. (7.88 ticks)
Probing time = 0.00 sec. (0.26 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search 

# 5

Organism: Mycobacterium tuberculosis

Version: iNJ661m

Year: 2010

Paper link: https://bmcsystbiol.biomedcentral.com/articles/10.1186/1752-0509-4-160

Model Link: https://static-content.springer.com/esm/art%3A10.1186%2F1752-0509-4-160/MediaObjects/12918_2010_571_MOESM2_ESM.XML

In [12]:
model =parseSBML('/home/hzabeti/Desktop/UCSD models/12918_2010_571_MOESM2_ESM.XML')

In [13]:
model.description

{'ID': 'iNJ661m_SBML_xml',
 'Name': 'iNJ661m_SBML_xml',
 'Source File': '/home/hzabeti/Desktop/UCSD models/12918_2010_571_MOESM2_ESM.XML'}

In [33]:
getSize(model.fullMatrix)

(934, 1049)

In [34]:
flm_index_list,flm_list = model.findFreeLunchMetabolites()

No Free Lunch condition: Not satisfied!
Optimum solution and the number of all possible free lunch metabolites 242


In [37]:
print(flm_index_list,flm_list)

60 M_4h2opntn_c


In [36]:
ind_flm_index_list,ind_flm_list = model.findIndividuallyProducibleFLM()

Number of individually producible free lunch metabolites 242


In [38]:
minimal_free_lunch,free_lunch_list,free_lunch_meta = model.findMinimalFreeLunch(free_lunch_meta_list=[60])

Number of iteration:  0
Number of reactions involved in FL: 31
------------------------------------------------------------
Checking index:  11
This index is in the minimal set 11
Checking index:  24
This index is in the minimal set 24
Checking index:  167
This index is in the minimal set 167
Checking index:  357
This index is in the minimal set 357
Checking index:  359
This index is in the minimal set 359
Checking index:  360
This index is in the minimal set 360
Checking index:  418
This index is in the minimal set 418
Checking index:  428
This index is in the minimal set 428
Checking index:  534
This index is in the minimal set 534
Checking index:  547
This index is in the minimal set 547
Checking index:  584
This index is in the minimal set 584
Checking index:  598
This index is in the minimal set 598
Checking index:  599
This index is in the minimal set 599
Checking index:  601
This index is in the minimal set 601
Checking index:  602
This index is in the minimal set 602
Checking i

In [15]:
flw_index_list, y_lower_bound = model.findFreeLunchWitnesses()

parameters.mip.tolerances.integrality
CPXPARAM_Simplex_Tolerances_Feasibility          1.0000000000000001e-09
CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Tolerances_Integrality              1.0000000000000001e-15
Found incumbent of value 508.000000 after 0.07 sec. (0.20 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 486 rows and 477 columns.
MIP Presolve modified 1583 coefficients.
Aggregator did 5 substitutions.
Reduced MIP has 1605 rows, 1499 columns, and 9448 nonzeros.
Reduced MIP has 884 binaries, 3 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.15 sec. (9.21 ticks)
Probing time = 0.00 sec. (0.18 ticks)
Tried aggregator 1 time.
Reduced MIP has 1605 rows, 1499 columns, and 9448 nonzeros.
Reduced MIP has 884 binaries, 3 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.21 sec. (3.53 ticks)
Probing time = 0.00 sec. (0.18 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, usi

# 6

Organism: Homo sapiens 

Version: Recon 2.2 

Year: 2016

Paper link: https://link.springer.com/article/10.1007%2Fs11306-016-1051-4

Model Link: https://github.com/mcisb/mcisb-recon-analysis/tree/master/models

In [16]:
model =parseSBML('/home/hzabeti/Desktop/UCSD models/recon_2.2.xml')

0) Generic human biomass reaction 1) protein component of biomass 2) DNA component of biomass 3) RNA component of biomass 4) carbohydrate component of biomass 5) lipid component of biomass 6) other component of biomass
Please select the correct string from the list above and enter its number, or -1 if none 
0


In [8]:
model.description

{'ID': 'recon2_2',
 'Name': 'Recon 2.2',
 'Source File': '/home/hzabeti/Desktop/UCSD models/recon_2.2.xml'}

In [9]:
getSize(model.fullMatrix)

(6047, 7785)

In [11]:
flm_index_list,flm_list = model.findFreeLunchMetabolites()

No Free Lunch condition: Satisfied!


In [18]:
flw_index_list, y_lower_bound = model.findFreeLunchWitnesses()

parameters.mip.tolerances.integrality


CPLEX Error  1222: Duplicate entry or entries.


CplexSolverError: CPLEX Error  1222: Duplicate entry or entries.

In [22]:
model.printReactionFormula(0)

AssertionError: 

Organism:

Version: 

Year:

Paper link: 

Model Link: 

In [19]:
import gc
gc.collect()

253727